In [1]:
import numpy as np
from skimage.transform import resize
import pylidc as pl
from unet import UNet

Using TensorFlow backend.


In [2]:
scans = pl.query(pl.Scan)
n = scans.count()

In [3]:
image_size = (256, 256)

In [4]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([im.pixel_array for im in images])
    X = resize(X, (n, *image_size), mode='constant')
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    y = np.expand_dims(y, axis=-1)
    return X, y

In [5]:
model = UNet((*image_size, 1), batchnorm=True, dropout=False)
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
for _ in range(25):
    i = np.random.randint(n)
    print(i)
    try:
        X, y = get_data(i)
        model.fit(X, y, batch_size=1, epochs=1)
    except OSError:
        print('Problem getting data')